# IOH-Xplain analyse example

This example assumes you have run cma_es_run-configurations.py and cma_es_process_pkl.py first.

In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from IPython.display import display
from config import cma_features, cmaes_explainer

data_file = "cma_final_processed.pkl"
features = cma_features
cmaes_explainer.load_results(data_file)
#use aucLarge for D30
cmaes_explainer.df.loc[cmaes_explainer.df["dim"] == 30,'auc'] = cmaes_explainer.df.loc[cmaes_explainer.df["dim"] == 30,'aucLarge']
df = cmaes_explainer.performance_stats()
display(df.style.bar(cmap='viridis'))


/home/neocortex/repos/ioh-explainer/env/lib/python3.8/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/neocortex/repos/ioh-explainer/env/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/neocortex/repos/ioh-explainer/env/lib/python3.8/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/neocortex/repos/ioh-explainer/env/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [2]:
# Now show behaviour statistics
df = cmaes_explainer.behaviour_stats(per_fid=False)
display(df.style.bar(cmap='viridis'))

In [2]:

#With the following command we can create a latex report of the performances per function id.
#cmaes_explainer.to_latex_report(include_explain=False, filename="cma_tables.tex")

In [3]:
#Correlation of configuration ranking per function
import seaborn as sns
import matplotlib.pyplot as plt


df = cmaes_explainer.df
for dim in cmaes_explainer.dims:
    fid_auc_matrix = []
    for fid in cmaes_explainer.fids:
        df_dim_fid = df[(df['dim']==dim) & (df['fid'] == fid)]
        
        fid_auc_matrix.append(df_dim_fid.groupby(features)['auc'].mean())

    fid_auc_matrix = np.array(fid_auc_matrix).T

    df_matrix = pd.DataFrame(fid_auc_matrix, columns=[f"$f_{{{i}}}$" for i in np.arange(1,25)])
    corr = df_matrix.corr()

    

    g = sns.clustermap(corr, 
                    method = 'complete', 
                    cmap   = 'viridis', 
                    annot  = True, 
                    annot_kws = {'size': 8})
    plt.tight_layout()
    plt.savefig(f"cma-es-clustermap-{dim}d.pdf")
    plt.clf()

    g = sns.heatmap(corr, 
                    annot  = False)
    plt.tight_layout()
    plt.savefig(f"cma-es-heatmap-{dim}d.pdf")
    plt.clf()
#plt.setp(g.ax_heatmap.get_xticklabels(), rotation=60);

The figure layout has changed to tight
The figure layout has changed to tight
The figure layout has changed to tight
The figure layout has changed to tight


<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

In [24]:
#Correlation of configuration ranking per function
df = cmaes_explainer.df
df_dim_fid = df[(df['dim']==5) & (df['fid'] == 2) & (df['seed'] == 1) & (df['iid'] == 1)]
df_dim_fid

,fid,iid,dim,seed,elitist,mirrored,base_sampler,weights_option,local_restart,step_size_adaptation,auc
4871,2,1,5,1,False,mirrored,gaussian,default,BIPOP,psr,9037.037037
7251,2,1,5,1,False,mirrored,sobol,default,IPOP,csa,9672.839506
9631,2,1,5,1,False,mirrored,halton,equal,BIPOP,csa,9179.012346
12011,2,1,5,1,False,mirrored pairwise,gaussian,default,NaN,psr,9141.975309
16771,2,1,5,1,True,NaN,halton,default,IPOP,psr,9456.790123
...,...,...,...,...,...,...,...,...,...,...,...
753381,2,1,5,1,True,mirrored,sobol,equal,NaN,csa,9703.703704
760521,2,1,5,1,True,mirrored,halton,equal,BIPOP,psr,9697.530864
761711,2,1,5,1,False,mirrored pairwise,gaussian,equal,IPOP,csa,2777.777778
762901,2,1,5,1,False,NaN,gaussian,1/2^lambda,IPOP,psr,9395.061728
